**Inputs**
* ImageCollection(s) containing hazard magnitudes for each year-model pair

**Outputs**
* ImageCollection(s) containing hazard probabilities for 2030 and 2065
* **You must manually create the empty ImageCollection** in the dataportal/naive-probs directory before trying to export results

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWhXJuxpo5loxZmF3DOFMpNGXOdM_N-4MRYyR63PIMBU9aqNisqQXjM

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
hazardnames = ['ds', 'maxdryspell']
hazard_info = {
    'ds': {
        'num_bins': 100,
        'bin_width': 1
    },
    'maxdryspell': {
        'num_bins': 73,
        'bin_width': 5
    }
}

In [4]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
EARLY_YEAR = 2030
LATE_YEAR = 2065

In [9]:
def make_naiveprobs_image(year):
    result_list = ee.List([])
    for bin_idx in range(hazard_info[hazardname]['num_bins']):
        model_year_list = ee.List([])
        mag_low = (bin_idx) * hazard_info[hazardname]['bin_width']
        mag_high = (bin_idx + 1) * hazard_info[hazardname]['bin_width']
        for y in range(year - 10, year + 10):
            for model in MODELS:
                modelyear_magnitudes = magnitudes.filterMetadata('model', 'equals', model).filterMetadata('year', 'equals', year).first()
                model_year_list = model_year_list.add(modelyear_magnitudes.gte(mag_low).multiply(1).multiply(modelyear_magnitudes.lt(mag_high).multiply(1)))
            mag_list = ee.ImageCollection(model_year_list).mean()
        result_list = result_list.add(mag_list.set('magnitude_low', mag_low).set('magnitude_high', mag_high))
    result_img = ee.ImageCollection(result_list).toBands().rename(['magnitude_{0}-{1}'.format(str(i * hazard_info[hazardname]['bin_width']), str((i+1) * hazard_info[hazardname]['bin_width'])) for i in range(hazard_info[hazardname]['num_bins'])])
    return result_img

In [10]:
for hazardname in hazardnames:
    magnitudes = ee.ImageCollection('users/tedwongwri/dataportal/magnitudes/' + hazardname)
    for year in [EARLY_YEAR, LATE_YEAR]:
        img = make_naiveprobs_image(year)
        task = ee.batch.Export.image.toAsset(**{
          'image': img.set('year', year),
          'description': '{0}_{1}'.format(hazardname, str(year)),
          'assetId': 'users/tedwongwri/dataportal/naive-probs/{0}_{1}'.format(hazardname, str(year)),
          'region': WHOLE_GLOBE,
          'crs': 'EPSG:4326',
          'dimensions': '1440x720'
        })
        task.start()